# DBus Getting started
The goal of this trainin is to know how to work with Dbus, so we will follow thoses steps:
1- Connect to system Dbus object and write/read values
2- Publish a custom event using dbus and read/write values

## Basics
The following demonstration was run in Ubuntu 16.04
* Connection to the Bus:
Applications that use D-Bus typically connect to a (session or system) bus daemon, which forwards messages between the applications. To use D-Bus, you need to create a Bus object representing the connection to the bus daemon.
First of all install the necessary dependencies:
~~~
sudo apt-get install libdbus-1-dev
sudo apt-get install libdbus-glib-1-dev
sudo pip install dbus-python
sudo apt-get install python-dbus
sudo apt-get install python3-dbus #for python 3
~~~

In [1]:
import dbus

In [2]:
session_bus = dbus.SessionBus() # to get session bus
system_bus = dbus.SystemBus() # get system bus

* Making method calls
D-Bus applications can export objects for other applications' use. To start working with an object in another application, you need to know:
- The bus name such as org.freedesktop.NetworkManager
- The object path. Applications can export many objects - for instance. As you'd expect, one of the main things you can do with remote objects is to call their methods. As in Python, methods may have parameters, and they may return one or more values.

## Connect to a system Dbus object and read values
To interact with a remote object, you use a proxy object. This is a Python object which acts as a proxy or "stand-in" for the remote object - when you call a method on a proxy object, this causes dbus-python to make a method call on the remote object, passing back any return values from the remote object's method as the return values of the proxy method call.
For example, NetworkManager has the well-known name org.freedesktop.NetworkManager and exports an object whose object path is /org/freedesktop/NetworkManager. 
To can get a proxy for the object representing the network manager we must do :

In [3]:
proxy = system_bus.get_object('org.freedesktop.NetworkManager', '/org/freedesktop/NetworkManager')

After that we retreive the manager or the interface of this proxy

In [5]:
manager = dbus.Interface(proxy, "org.freedesktop.NetworkManager")

After we can get devices's specific state :

In [17]:
import socket, struct
devices = manager.GetDevices()
for device in devices:
    device_proxy = system_bus.get_object("org.freedesktop.NetworkManager", device)
    iface_properties = dbus.Interface(device_proxy, 
                                      "org.freedesktop.DBus.Properties")
    # Get the devices's current state and interface name
    state = iface_properties.Get("org.freedesktop.NetworkManager.Device", 
                                "State")
    
    name = iface_properties.Get("org.freedesktop.NetworkManager.Device",
                               "Interface")
    ifa = "org.freedesktop.NetworkManager.Device"
    type = iface_properties.Get(ifa, "DeviceType")
    addr = iface_properties.Get(ifa, "Ip4Address")
    
    # print the result
    addr_dotted = socket.inet_ntoa(struct.pack('<L', addr))
    s = "Devices %s is activated and has type %s and address %s"
    print(s % (name, type, addr_dotted))

Devices wlp2s0 is activated and has type 2 and address 10.10.0.222
Devices lo is activated and has type 14 and address 127.0.0.1
Devices virbr0 is activated and has type 13 and address 192.168.122.1
Devices virbr0-nic is activated and has type 16 and address 0.0.0.0


## Write value to existing DBus object
Lets send notification to user in its Desktop throw the org.freedesktop.Notifications object:

In [20]:
item              = "org.freedesktop.Notifications"
path              = "/org/freedesktop/Notifications"
interface         = "org.freedesktop.Notifications"
app_name          = "My Great Application"
id_num_to_replace = 0
icon              = "/usr/share/icons/cab_view.png"
title             = "Notification Title"
text              = "This is the body"
actions_list      = ''
hint              = ''
time              = 5000   # Use seconds x 1000

notify_proxy = session_bus.get_object(item, path)
notify_interface = dbus.Interface(notify_proxy, interface)
notify_interface.Notify(app_name, id_num_to_replace, icon, 
                        title, text, actions_list, hint, time)

dbus.UInt32(3L)

the result should look like this:

![Notification result](notify_result.png)


## Introcpecting an object:


In [30]:
nmanager_object = system_bus.get_object(
    'org.freedesktop.NetworkManager',          # service
    '/org/freedesktop/NetworkManager'  # published object
)
introspection_interface = dbus.Interface(
    nmanager_object,
    dbus.INTROSPECTABLE_IFACE,
)
# Introspectable interfaces define a property 'Introspect' that
# will return an XML string that describes the object's interface
interface = introspection_interface.Introspect()
print(interface)

<!DOCTYPE node PUBLIC "-//freedesktop//DTD D-BUS Object Introspection 1.0//EN"
                      "http://www.freedesktop.org/standards/dbus/1.0/introspect.dtd">
<!-- GDBus 2.48.2 -->
<node>
  <interface name="org.freedesktop.DBus.Properties">
    <method name="Get">
      <arg type="s" name="interface_name" direction="in"/>
      <arg type="s" name="property_name" direction="in"/>
      <arg type="v" name="value" direction="out"/>
    </method>
    <method name="GetAll">
      <arg type="s" name="interface_name" direction="in"/>
      <arg type="a{sv}" name="properties" direction="out"/>
    </method>
    <method name="Set">
      <arg type="s" name="interface_name" direction="in"/>
      <arg type="s" name="property_name" direction="in"/>
      <arg type="v" name="value" direction="in"/>
    </method>
    <signal name="PropertiesChanged">
      <arg type="s" name="interface_name"/>
      <arg type="a{sv}" name="changed_properties"/>
      <arg type="as" name="invalidated_propertie